In [1]:
import numpy as np
from scipy import sparse
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
class PegasosSVM:
    def __init__(self, lambda_reg=0.01, iterations=10000):
        self.lambda_reg = lambda_reg
        self.iterations = iterations
        self.w = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        
        # SVM için etiketler -1 ve 1 olmalı [cite: 32-35]
        y_encoded = np.where(y <= 0, -1, 1).ravel()

        for t in range(1, self.iterations + 1):
            i = np.random.randint(0, n_samples)
            xi = X[i]
            yi = y_encoded[i]

            eta = 1.0 / (self.lambda_reg * t)

            # Karar değeri (margin) hesabı
            if sparse.issparse(xi):
                margin = float(xi.dot(self.w).item())
            else:
                margin = np.dot(xi, self.w)

            # L2 regularization (sönümleme) her adımda yapılır [cite: 35]
            self.w *= (1 - eta * self.lambda_reg)

            # Hinge loss şartı: yi * f(xi) < 1 ise gradyan ekle [cite: 34]
            if yi * margin < 1:
                if sparse.issparse(xi):
                    # Sadece non-zero indeksleri güncelleyerek hız kazanıyoruz [cite: 37]
                    self.w[xi.indices] += eta * yi * xi.data
                else:
                    self.w += eta * yi * xi

            # Projeksiyon adımı (Ağırlıkların aşırı büyümesini engeller)
            norm = np.linalg.norm(self.w)
            bound = 1.0 / np.sqrt(self.lambda_reg)
            if norm > bound:
                self.w *= (bound / norm)

    def predict(self, X):
        # Karar mesafelerini hesapla ve 0/1 formatına döndür
        if sparse.issparse(X):
            scores = X.dot(self.w)
        else:
            scores = np.dot(X, self.w)
        return np.where(scores >= 0, 1, 0)
    
    def predict_scores(self, X):
        # Görev 3'te (Stacking) kullanılacak ham skorlar [cite: 41]
        if sparse.issparse(X):
            return X.dot(self.w)
        return np.dot(X, self.w)

In [3]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np

# HIGGS Veri kümesini yükleyen hücre 
X_train = np.load("X_train_higgs.npy")
y_train = np.load("y_train_higgs.npy")
X_test  = np.load("X_test_higgs.npy")
y_test  = np.load("y_test_higgs.npy")
# Modeli oluşturma (Hyperparametreleri ödev analizine göre değiştirebilirsiniz)
svm_higgs = PegasosSVM(lambda_reg=0.01, iterations=20000)

# Eğitim (HIGGS)
print("HIGGS SVM Eğitiliyor...")
svm_higgs.fit(X_train, y_train)

# Test ve Değerlendirme
y_pred_higgs = svm_higgs.predict(X_test)

print("\n--- HIGGS SVM SONUÇLARI ---")
print("Accuracy Score:", accuracy_score(y_test, y_pred_higgs))
print("F1 Score      :", f1_score(y_test, y_pred_higgs))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_higgs))

HIGGS SVM Eğitiliyor...

--- HIGGS SVM SONUÇLARI ---
Accuracy Score: 0.62125
F1 Score      : 0.6532387273975738
Confusion Matrix:
 [[1058  838]
 [ 677 1427]]


In [11]:
from sklearn.datasets import fetch_rcv1
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from scipy import sparse

# 1. RCV1 Veri Kümesini Yükleme ve Hazırlama
print("RCV1 veri kümesi indiriliyor/yükleniyor...")
rcv1 = fetch_rcv1()
X_rcv = rcv1.data  # Bu bir CSR sparse matristir
y_rcv = rcv1.target

# Ödev gereği 'CCAT' kategorisini (indeks 33) hedef değişken yapıyoruz
# CCAT, rcv1.target_names içinde 33. sırada bulunur
ccat_idx = 33 
y_ccat = y_rcv[:, ccat_idx].toarray().ravel() # 0 ve 1 değerlerine sahip dizi

# ÖNEMLİ: Bellek sorunu yaşamamak için veri setinden bir örneklem alalım (Örn: 20.000 satır)
# Bilgisayarın güçlüyse bu sayıyı artırabilirsin
subset_size = 20000
indices = np.random.choice(X_rcv.shape[0], subset_size, replace=False)
X_subset = X_rcv[indices]
y_subset = y_ccat[indices]

# Veriyi Eğitim ve Test olarak ayırma
X_train_rcv1, X_test_rcv1, y_train_rcv1, y_test_rcv1 = train_test_split(
    X_subset, y_subset, test_size=0.2, random_state=42
)

print(f"RCV1 Hazır! Eğitim boyutu: {X_train_rcv1.shape}")

# 2. Pegasos SVM Modelini Eğitme (Daha önce yazdığın sınıfı kullanır)
# RCV1 çok boyutlu olduğu için iterasyon sayısını yüksek tutmak faydalı olabilir
svm_rcv1 = PegasosSVM(lambda_reg=0.001, iterations=50000)

print("RCV1 üzerinde SVM (Pegasos) eğitiliyor...")
svm_rcv1.fit(X_train_rcv1, y_train_rcv1)

# 3. Test ve Değerlendirme
y_pred_rcv1 = svm_rcv1.predict(X_test_rcv1)

print("\n--- RCV1 SVM PERFORMANS SONUÇLARI ---")
print(f"Accuracy: {accuracy_score(y_test_rcv1, y_pred_rcv1):.4f}")
print(f"F1-Score: {f1_score(y_test_rcv1, y_pred_rcv1):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test_rcv1, y_pred_rcv1))


RCV1 veri kümesi indiriliyor/yükleniyor...
RCV1 Hazır! Eğitim boyutu: (16000, 47236)
RCV1 üzerinde SVM (Pegasos) eğitiliyor...

--- RCV1 SVM PERFORMANS SONUÇLARI ---
Accuracy: 0.9197
F1-Score: 0.9125
Confusion Matrix:
[[2005   81]
 [ 240 1674]]
